In [1]:
import numpy as np
from collections import defaultdict, Counter

class MarkovModel:
    
    def __init__(self, text, k):
        '''
        create a markov model of order k from griven text
        assume that text has length at least k
        '''
        self.k = k
        self.transition_count = defaultdict(float) # number of times each char follows each kgram
        self.alphabet = list(set(list(text)))      # alphabet taken from the text
        self.char_counts = Counter(text)           # number of times each char appears in the text
        self.kgrams = defaultdict(int)             # number of times each kgram appears in the text
        self.n = len(text)                         # length of the text
        text += text[:k]                           # wrap the text (i.e. make it 'circular')
        for i in range(self.n):
            self.transition_count[text[i:i + k], text[i + k]] += 1
            self.kgrams[text[i:i + k]] += 1
    
    def order(self):
        '''
        order k of Markow model
        '''
        return self.k
    
    def freq(self, kgram):
        '''
        number of occurences of kgram in text
        ''' 
        assert len(kgram) == self.k, "length, {}, of given kgram does not match the order, {}, of this Markov model.".format(len(kgram), self.k)
        return self.kgrams[kgram]
    
    def freq2(self, kgram, c):
        '''
        number of times that character c follows kgram
        '''
        assert len(kgram) == self.k, "length, {}, of given kgram does not match the order, {}, of this Markov model.".format(len(kgram), self.k)
        return self.transition_count[kgram, c]
        
    def prob(self, kgram, c):
        '''
        probability that character c follows kgram
        '''
        assert len(kgram) == self.k, "length, {}, of given kgram does not match the order, {}, of this Markov model.".format(len(kgram), self.k)
        N = sum([self.transition_count[kgram, letter] for letter in self.alphabet])
        if N == 0:
            # there are no instances of the string kgram in the text
            # therefore use the distribution of chars in the text to return 
            # a probability
            return self.char_counts[c]/self.n
        else:
            # use the distribution given by existing transitions
            return self.transition_count[kgram, c]/N
    
    def rand(self, kgram):
        '''
        random character following given kgram
        '''
        assert len(kgram) == self.k, "length, {}, of given kgram does not match the order, {}, of this Markov model.".format(len(kgram), self.k)
        return np.random.choice(self.alphabet, 1, p=np.array([self.prob(kgram, letter) for letter in self.alphabet]))[0]
    
    def gen(self, kgram, T):
        '''
        generate a string of length T characters using kgram as a seed
        '''
        assert len(kgram) == self.k, "length, {}, of given kgram does not match the order, {}, of this Markov model.".format(len(kgram), self.k)
        
        string = ''
        for _ in range(T):
            # generate a random letter to follow kgram
            c = self.rand(kgram)
            
            # slide the window
            kgram = kgram[1:] + c
            
            # add c to the string generated so far
            string += c
            
        return string

    def most_likely(self, context):
        '''
        return the most likely letter, given the surrounding letters in context
        '''
        assert len(context) == 2 * self.k + 1, "the length of the context is invalid, context should have length {}".format(2 * self.k + 1)
        
        most_likely_char = self.alphabet[0]
        
        log_likelihood = defaultdict(float)
        for c in self.alphabet:
            # replace the corrupted char with c
            test_context = context[:self.k] + c + context[self.k + 1:]
            
            # compute the log-likelihood of each window given c as the replacement char
            # NOTE: we take log(P + 0.01) instead of log(P) to avoid
            #       trouble with dividing by 0
            probs = [np.log(self.prob(test_context[i:i + self.k], test_context[i + self.k]) + 0.01) for i in range(self.k)]
            
            # store the log-likelihood for c
            log_likelihood[c] = sum(probs)
            
            # compare this log-likelihood with the log-likelihood of the current argmax
            if log_likelihood[c] > log_likelihood[most_likely_char]:
                most_likely_char = c
                
        return most_likely_char
    
    def replace_unknown(self, corrupted):
        '''
        replace unknown characters with most probable characters
        '''
        corrected = corrupted
        unknown_idx = corrupted.find('~')
        while unknown_idx > -1:
            corrected = corrected[:unknown_idx] + self.most_likely(corrected[unknown_idx - self.k: unknown_idx + self.k + 1]) + corrected[unknown_idx + 1:]
            unknown_idx = corrupted.find('~', unknown_idx + 1)
        return corrected

---

In [2]:
m = MarkovModel('gagggagaggcgagaaa', 3)

In [3]:
m.gen('gag', 9)

'gcgagaggc'

In [4]:
m.prob('gag', 'c')

0.0

In [5]:
m.most_likely('agg~gag')

'c'

---

In [6]:
text = '''Microsoft said Tuesday the company would comply with a preliminary ruling by Federal District Court Judge Ronald H. Whyte that Microsoft is no longer able to use the Java Compatibility Logo on its packaging and websites for Internet Explorer and Software Developers Kit for Java. 

“We remain confident that once all the facts are presented in the larger case, the court will find Microsoft to be in full compliance with its contract with Sun,” stated Tom Burt, Associate General Counsel for Microsoft Corporation. “We are disappointed with this decision, but we will immediately comply with the Court’s order.” 

Microsoft has been in the forefront of helping developers use the Java programming language to write cutting-edge applications. The company has committed significant resources so that Java developers have the option of taking advantage of Windows features when writing software using the Java language. Providing the best tools and programming options will continue to be Microsoft’s goal. 

“We will continue to listen to our customers and provide them the tools they need to write great software using the Java language,” added Tod Nielsen, General Manager for Microsoft’s Developer Relations Group/Platform Marketing.'''

In [7]:
order = 7
m = MarkovModel(text, order)
seed = text[:order]
print(seed + m.gen(seed, 1000))

Microsoft said Tuesday the company would comply with this decision, but we will continue to listen to our customers and provide them the tools and provide them the tools and programming language,” added Tod Nielsen, General Counsel for Microsoft’s Developers have the option of taking advantage of Windows features when writing software using the Java programming options will continue to be Microsoft said Tuesday the court will find Microsoft is no longer able to use the Java programming option of taking advantage of Windows features when writing software using the Java language,” added Tod Nielsen, General Counsel for Microsoft said Tuesday the court will find Microsoft to be Microsoft’s goal. 

“We remain confident that once all the facts are presented in the larger case, the company would comply with the Court’s order.” 

Microsoft Corporation. “We are disappointed with this decision, but we will immediately comply with the Court’s order.” 

Microsoft said Tuesday the court will find 

---

In [8]:
m = MarkovModel("it was the best of times, it was the worst of times.", 4)
m.replace_unknown("it w~s th~ bes~ of tim~s, i~ was ~he wo~st of~times.")

'it was the best of times, it was the worst of times.'